In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [ ]:
lines=pd.read_csv('/content/drive/MyDrive/translator/Hindi_English_Truncated_Corpus.csv',encoding='utf-8')

In [ ]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [ ]:
lines=lines[lines['source']=='ted']

In [ ]:
lines.head(20)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
8545,ted,she was chased from her home and had to find a...,START_ उसे घर से भगा दिया गया था और उसे दूसरा ...,13,15
25292,ted,and have this kind of multiresolution experience,START_ और इस प्रकार के बहु अभिवेधन का अनुभव कर...,7,13
13636,ted,in i was in bangladesh,START_ से तक मैं बाँगलादेश में था _END,5,8
53614,ted,but its good enough to know what you dont want...,START_ मगर इतना भी पर्याप्त है कि पता हो कि आप...,12,16
84295,ted,is lean into the discomfort of the work,START_ काम की बेआरामी में समा जाओ _END,8,8
18442,ted,and he said “well okay”,START_ उन्होंने कहा “अच्छा ठीक है” _END,5,7
41016,ted,or alternatively we invoke thoreau,START_ या फिर हम थोरेयू का आह्वान करते हैं _END,5,10
117868,ted,the hippocampus is formed of two sheets of cells,START_ हिप्पोकैम्पस कोशिकाओं की दो परतों से बन...,9,10
7184,ted,and this is a therapist who sees therapists,START_ और ये एक ऐसी थेरेपिस्ट है जो थैरेपिस्टो...,8,14
44855,ted,thankfully baby was out mother and child survived,START_ शुक्र है शिशु बहार आ गया जच्चाबच्चा दोन...,8,13


In [ ]:

pd.isnull(lines).sum()

source                 0
english_sentence       0
hindi_sentence         0
length_eng_sentence    0
length_hin_sentence    0
dtype: int64

In [ ]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [ ]:

lines.drop_duplicates(inplace=True)

In [ ]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 5)

In [ ]:

lines.shape

(25000, 5)

In [ ]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [ ]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.head()


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
11355,ted,pranitha shaheen and anjali,START_ start प्रणिता शाहीन और अंजलि end _END,4,6
104589,ted,people were trying to do them,START_ start लोग कोशिश कर रहे थे end _END,6,7
89468,ted,its very easy to lose perspective,START_ start तो अपना आपा खोना बहुत आसान होता ह...,6,10
92165,ted,to take the photo,START_ start इस फोटो के लिए end _END,4,6
79264,ted,to sound its music in the ear and on the tongue,START_ start ताकि इसका संगीत कानों और ज़बान को...,11,11


In [ ]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [ ]:
len(all_eng_words)

13948

In [ ]:
len(all_hindi_words)

17539

In [ ]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
11355,ted,pranitha shaheen and anjali,START_ start प्रणिता शाहीन और अंजलि end _END,4,8
104589,ted,people were trying to do them,START_ start लोग कोशिश कर रहे थे end _END,6,9
89468,ted,its very easy to lose perspective,START_ start तो अपना आपा खोना बहुत आसान होता ह...,6,12
92165,ted,to take the photo,START_ start इस फोटो के लिए end _END,4,8
79264,ted,to sound its music in the ear and on the tongue,START_ start ताकि इसका संगीत कानों और ज़बान को...,11,13


In [ ]:
lines[lines['length_eng_sentence']>30].shape

(0, 5)

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [ ]:
lines.shape

(24585, 5)

In [ ]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [ ]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(13948, 17539)

In [ ]:

num_decoder_tokens += 1 #for zero padding

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
61228,ted,breast cancer for the love of lebanon for peace,START_ start स्तन कैंसर लेबनान के प्यार के लिए...,9,14
38735,ted,so the empire can strike back,START_ start तो अब साम्राज्य जवाबी हमला कर सकत...,6,12
1521,ted,that if nectar,START_ start अगर पराग end _END,3,6
45887,ted,what we see changes who we are,START_ start जो हम देखते हैं वो हमें बदल डालता...,7,13
102726,ted,and so netflix has gone through,START_ start और नेत्फ्लिक्स ने देखे हैं end _END,6,9
112391,ted,and there are lots of physical parameters that...,START_ start और बहुत सारे भौतिक मापदंड हैं जो ...,9,13
64393,ted,as people use it and whose benefits become gre...,START_ start और लोगों के लिये प्रयोग करने के ब...,9,12
79818,ted,this idea of compassion comes to us,START_ start हमें करुणा का विचार आता है end _END,7,10
90523,ted,and why what is wrong with his underwear”,START_ start और किसलिए क्या खराबी है उनके कच्छ...,8,12
48695,ted,inside they are selling for pounds dollars,START_ start ये लोग सौ दो सौ गुना ज़्यादा महँग...,7,15


In [ ]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19668,), (4917,))

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim=400

In [ ]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, activation='tanh')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, activation='tanh')
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
from keras.optimizers import Adam

# Set your desired learning rate
learning_rate = 0.001

# Define your optimizer with the specified learning rate
optimizer = Adam(learning_rate=learning_rate)

# Compile your model with the optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 400)            5579200   ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, None, 400)            7016000   ['input_4[0][0]']             
                                                                                            

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)
model.save('translate.h5')

<ipython-input-80-314dbca857c5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100
153/153 [==============================] - 80s 463ms/step - loss: 5.8579 - accuracy: 0.2379 - val_loss: 5.0952 - val_accuracy: 0.2977
Epoch 2/100
153/153 [==============================] - 54s 354ms/step - loss: 4.8433 - accuracy: 0.3138 - val_loss: 4.8554 - val_accuracy: 0.3285
Epoch 3/100
153/153 [==============================] - 54s 353ms/step - loss: 4.5837 - accuracy: 0.3374 - val_loss: 4.7700 - val_accuracy: 0.3355
Epoch 4/100
153/153 [==============================] - 62s 408ms/step - loss: 4.4095 - accuracy: 0.3476 - val_loss: 4.6966 - val_accuracy: 0.3412
Epoch 5/100
153/153 [==============================] - 61s 401ms/step - loss: 4.2386 - accuracy: 0.3568 - val_loss: 4.6434 - val_accuracy: 0.3466
Epoch 6/100
153/153 [==============================] - 53s 347ms/step - loss: 4.0862 - accuracy: 0.3676 - val_loss: 4.6205 - val_accuracy: 0.3523
Epoch 7/100
153/153 [==============================] - 53s 345ms/step - loss: 3.9405 - accuracy: 0.3793 - val_loss: 4.6103 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

In [ ]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: but that slowness that caution
Actual Hindi Translation:  start लेकिन यहसुस्ती यह सावधानी end 
Predicted Hindi Translation:  start लेकिन यहसुस्ती यह सावधानी end 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: are bad at paying attention
Actual Hindi Translation:  start ध्यान नहीं दे पाते end 
Predicted Hindi Translation:  start ध्यान नहीं दे पाते end 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: we have reached a moment from which can emerge
Actual Hindi Translation:  start वो क्षण आ गया है जिससे उभर सकता है end 
Predicted Hindi Translation:  start वो क्षण आ गया है जिससे उभर सकता है end 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 18ms/step
Input English sentence: the first was the feedback from my cousins
Actual Hindi Translation:  start सबसे पहले तो मेरे रिश्तेदारों की प्रकिया। end 
Predicted Hindi Translation:  start सबसे पहले तो मेरे रिश्तेदारों की प्रकिया।


In [ ]:

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 18ms/step
Input English sentence: and so if we can do that then we can free up
Actual Hindi Translation:  start इस प्रकार यदि हम इस कार्य में सफल रहे तो end 
Predicted Hindi Translation:  start इस प्रकार यदि हम इस कार्य में सफल रहे तो


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Your hypothesis and reference strings
a = y_train[k:k+1].values[0][6:-4]
b = decoded_sentence[:-4]

# Calculate BLEU score with smoothing
score = sentence_bleu([b], a, smoothing_function=SmoothingFunction().method1)
print('Bleu score:', '%.6f' % score)


Bleu score: 0.900939
